In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel
from sklift.models import TwoModels

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklift.datasets import fetch_lenta
from sklearn.model_selection import train_test_split


from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline

import dill
from flask import Flask, request, jsonify

In [2]:
dill._dill._reverse_typemap['ClassType'] = type

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")
treat_test = pd.read_csv('treat_test.csv')

In [3]:
X_test.head()

,gender,age,main_format,children,crazy_purchases_goods_count_6m,k_var_cheque_group_width_15d,promo_share_15d
0,Ж,44.0,0,1.0,0.0,NaN,0.3727
1,Ж,56.0,0,2.0,2.0,0.0000,0.0000
2,Ж,41.0,0,0.0,0.0,0.0000,0.0000
3,М,27.0,0,0.0,0.0,0.6600,0.5260
4,Ж,29.0,0,0.0,2.0,0.3111,0.5533


In [4]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226720 entries, 0 to 226719
Data columns (total 7 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   gender                          223831 non-null  object 
 1   age                             222793 non-null  float64
 2   main_format                     226720 non-null  int64  
 3   children                        223831 non-null  float64
 4   crazy_purchases_goods_count_6m  226719 non-null  float64
 5   k_var_cheque_group_width_15d    181586 non-null  float64
 6   promo_share_15d                 226720 non-null  float64
dtypes: float64(5), int64(1), object(1)
memory usage: 12.1+ MB


In [5]:
import requests

import urllib.request
import json      


def send_json(x):
    gender, age, main_format, children, crazy_purchases_goods_count_6m, k_var_cheque_group_width_15d, promo_share_15d  = x
    
    body = {'gender': gender, 
            'age': age, 
            'main_format': int(main_format), 
            'children': children, 
            'crazy_purchases_goods_count_6m': crazy_purchases_goods_count_6m, 
            'k_var_cheque_group_width_15d': k_var_cheque_group_width_15d,
            'promo_share_15d': promo_share_15d
            } 

    myurl = "http://127.0.0.1:5000" + "/predict"
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)

    return response.json()['predictions']

In [6]:
response = send_json(X_test.iloc[0,:])

response

0.0030528900486946065

In [9]:
X_test.shape

(226720, 7)

In [11]:
N = 1000
predictions = X_test.iloc[:N].apply(lambda x: send_json(x), 1)

In [13]:
sm_30 = uplift_at_k(y_test.iloc[:N], predictions, treat_test.iloc[:N], strategy='by_group')
print(f'uplift@30%: {sm_30:.4f}')

uplift@30%: 0.0291
